In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
data = pd.read_csv('../Data/CVD_cleaned.csv')

In [ ]:
display(data.head())

In [ ]:
# TODO
# visualise interesting / important variables (or combinations thereof)

In [ ]:
# reformatting the data to be suitable for logistic regression
# transforming categorical variables (Yes=1, No=0)
# transforming 'Sex' variable to binary 'Sex_Male' variable
data = data.replace({'Skin_Cancer':'Yes', 'Other_Cancer': 'Yes', 'Heart_Disease': 'Yes', 'Depression': 'Yes', 'Smoking_History': 'Yes', 'Exercise': 'Yes'
                     , 'Sex': 'Male', 'Arthritis': 'Yes'}, 1).replace({'Skin_Cancer':'No', 'Other_Cancer': 'No', 'Heart_Disease': 'No'
                                                   , 'Depression': 'No', 'Smoking_History': 'No', 'Exercise': 'No'
                                                   , 'Sex': 'Female', 'Arthritis': 'No'}, 0).rename(columns={'Sex':'Sex_Male'})

In [ ]:
data.head()

In [ ]:
print(data['Age_Category'].unique())

In [ ]:
print(data['Diabetes'].unique())

In [ ]:
# TODO
# normalise numerical variables [0, 1]


In [ ]:
# TODO
# reformat multi-categorical variables:
# - Age (now needs to be normalised)
# - Diabetes (check if dropping all other options than 'Yes' and 'No' can be justified) 
# - General Health
# - Checkup
# either drop the consumption categories or find out what the values mean

# set age to numerical by taking the mean of the existing categories (ASSUMPTION: 80+ is estimated to 85)
age_mapping = {'18-24': 21.0, '25-29': 27.0, '30-34': 32.0, '35-39': 37.0,'40-44': 42.0, '45-49': 47.0, '50-54': 52.0, 
               '55-59': 57.0,'60-64': 62.0, '65-69': 67.0, '70-74': 72.0, '75-79': 77.0, '80+': 85.0}

# apply the mapping
data['Age_Category'] = data['Age_Category'].map(age_mapping)
data = data.rename(columns={'Age_Category':'Age_Numeric'})

# normalise the age
min_age = min(data['Age_Numeric'])
max_age = max(data['Age_Numeric'])

normalised_age = [(i - min_age) / (max_age - min_age) for i in data['Age_Numeric']]

data['Age_Numeric'] = normalised_age
data = data.rename(columns={'Age_Numeric':'Age_Normalised'})

# diabetes transformation
# group diabetics and pre-diabetics together, group non-diabetics and gestational diabetics together
diabetes_binary = [1 if i == 'Yes' or i == 'No, pre-diabetes or borderline diabetes' else 0 for i in data['Diabetes']]


In [ ]:
# TODO
# calculate/plot correlation coefficients

In [ ]:
Y = data['Heart_Disease']
X = data[['BMI', 'Weight_(kg)', 'Height_(cm)', 'Exercise', 'Depression', 'Smoking_History', 'Sex_Male', 'Skin_Cancer', 'Other_Cancer']]

In [ ]:
model = sm.Logit(Y, X).fit()

In [ ]:
model.summary()